# Data Vizualization

In [4]:
import sqlite3
import pandas as pd

# Establishing a connection to the SQLite database
conn = sqlite3.connect('/content/UEFA_ChampionsLeague_db_project.db')

# Querying the database
query = """
SELECT
    Matches.Match_ID AS MatchID,
    Matches.Season AS Season,
    Matches.Date_Time AS MatchDate,
    HomeTeam.Team_Name AS HomeTeam,
    AwayTeam.Team_Name AS AwayTeam,
    Matches.Home_Team_Score AS HomeScore,
    Matches.Away_Team_Score AS AwayScore,
    Tournaments.Tournament_Name AS Tournament,
    Tournaments.Year AS TournamentYear,
    Stadiums.Stadium_Name AS Stadium,
    Stadiums.City AS StadiumCity,
    Stadiums.Country AS StadiumCountry,
    Referees.Referee_Name AS Referee,
    Players.Player_Name AS Scorer,
    Players.Nationality AS ScorerNationality,
    Goals.Match_Time AS GoalTime,
    Goals.Goal_Desc AS GoalDescription
FROM Matches
LEFT JOIN Teams AS HomeTeam ON Matches.Home_Team_ID = HomeTeam.Team_ID
LEFT JOIN Teams AS AwayTeam ON Matches.Away_Team_ID = AwayTeam.Team_ID
LEFT JOIN Tournaments ON Matches.Tournament_ID = Tournaments.Tournament_ID
LEFT JOIN Stadiums ON Matches.Stadium_ID = Stadiums.Stadium_ID
LEFT JOIN Referees ON Matches.Referee_ID = Referees.Referee_ID
LEFT JOIN Goals ON Matches.Match_ID = Goals.Match_ID
LEFT JOIN Players ON Goals.Scored_ID = Players.Player_ID
ORDER BY Matches.Date_Time DESC;
"""

# Loading the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()


In [15]:
# Визуализация количества голов по командам
home_goals = df.groupby('HomeTeam')['HomeScore'].sum().reset_index()
away_goals = df.groupby('AwayTeam')['AwayScore'].sum().reset_index()
total_goals = pd.merge(home_goals, away_goals, left_on='HomeTeam', right_on='AwayTeam')
total_goals['TotalGoals'] = total_goals['HomeScore'] + total_goals['AwayScore']

fig1 = px.bar(total_goals, x='HomeTeam', y='TotalGoals',
             title='Количество голов по командам',
             labels={'HomeTeam': 'Команды', 'TotalGoals': 'Количество голов'},
             template='plotly')
fig1.show()

# Визуализация победителей турниров по годам
tournament_winners = df[['Tournament', 'TournamentYear', 'HomeTeam', 'HomeScore', 'AwayTeam', 'AwayScore']]
tournament_winners['Winner'] = tournament_winners.apply(lambda row: row.HomeTeam if row.HomeScore > row.AwayScore else row.AwayTeam, axis=1)

fig2 = px.histogram(tournament_winners, x='TournamentYear', color='Winner',
                   title='Победители турниров по годам',
                   labels={'TournamentYear': 'Год', 'Winner': 'Победители'},
                   template='plotly')
fig2.show()

# Топ игроков по количеству голов
top_scorers = df.groupby('Scorer')['GoalDescription'].count().reset_index()
top_scorers.columns = ['Player', 'Goals']
top_scorers = top_scorers.sort_values(by='Goals', ascending=False).head(10)

fig3 = px.bar(top_scorers, x='Goals', y='Player',
             title='Топ 10 игроков по количеству голов',
             labels={'Goals': 'Количество голов', 'Player': 'Игроки'},
             orientation='h', template='plotly')
fig3.show()

import plotly.express as px

# Распределение голов по времени матча
import plotly.express as px

# Распределение голов по времени матча
fig = px.histogram(
    df,
    x='GoalTime',
    nbins=30,  # Устанавливаем количество интервалов (bins)
    color='HomeTeam',  # Добавляем цвет для домашней команды
    title='Распределение голов по времени матча',
    labels={'GoalTime': 'Время голов (минуты)', 'HomeTeam': 'Команда'}
)

# Настройка осей
fig.update_layout(
    xaxis_title="Время голов (минуты)",
    yaxis_title="Количество голов",
    xaxis=dict(range=[0, 90])  # Ограничиваем ось X временем матча
)

fig.show()



<ipython-input-15-ba90e41444c4>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
# Распределение голов по времени матча  с группировкой по странам
fig = px.histogram(
    df,
    x='GoalTime',
    nbins=30,  # Устанавливаем количество интервалов (bins)
    color='ScorerNationality',  # Добавляем цвет по странам
    title='Распределение голов по времени матча',
    labels={'GoalTime': 'Время голов (минуты)', 'ScorerNationality': 'Страна'}
)

# Настройка осей
fig.update_layout(
    xaxis_title="Время голов (минуты)",
    yaxis_title="Количество голов",
    xaxis=dict(range=[0, 90])  # Ограничиваем ось X временем матча
)

fig.show()



In [9]:
# Группировка данных по сезону и суммарное количество голов
goals_by_season = df.groupby('Season')[['HomeScore', 'AwayScore']].sum().reset_index()
goals_by_season['TotalGoals'] = goals_by_season['HomeScore'] + goals_by_season['AwayScore']

# Построение гистограммы
fig = px.bar(
    goals_by_season,
    x='Season',
    y='TotalGoals',
    title='Общее число голов по сезонам',
    labels={'Season': 'Сезон', 'TotalGoals': 'Голы'},
    template='plotly',
    text_auto=True  # Отображение значений на графике
)
fig.update_traces()  # Цвет колонок
fig.show()


In [13]:
# Подготовка данных
team_goals = pd.merge(home_goals, away_goals, left_on='HomeTeam', right_on='AwayTeam', how='outer').fillna(0)
team_goals = team_goals.rename(columns={'HomeTeam': 'Team', 'HomeScore': 'HomeGoals', 'AwayScore': 'AwayGoals'})

# Упорядочение данных
team_goals = team_goals.sort_values(by='HomeGoals', ascending=False).head(15)

fig = px.bar(
    team_goals,
    x='Team',
    y=['HomeGoals', 'AwayGoals'],  # Две категории
    title='Сравнение домашних и выездных голов по командам',
    labels={'value': 'Голы', 'variable': 'Тип голов', 'Team': 'Команда'},
    barmode='group',
    template='plotly',
)
fig.update_layout(legend_title_text='Голы')
fig.show()
